<a href="https://colab.research.google.com/github/sangjinsu/personalized-recommendation-system/blob/main/MF_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>